In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2024-09-19


Found rundate 2024-09-18 10:00:02 < 2024-09-19 at commit c0ed096f; returning commit 9834d23


'2024-09-19 10:00:02'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2024-09-19 10:00:02, NJSP was reporting 471 YTD deaths
Current YTD Deaths (2025-09-19 10:00:01-04:00): 395
Previous year YTD Deaths (adjusted; 2024-09-19 10:00:02): 470.49033154731916
Projected 2025 total: 583.9778706593229
-16.0% change


45cc02e9ab99460dfe64fcd3e1a168b1b5d4db5e: FAUQStats cache miss: 2024, Thu Sep 19 10:00:02 EDT 2024


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

,CCODE,CNAME,MCODE,MNAME,STREET,LOCATION,FATALITIES,FATAL_D,FATAL_P,FATAL_T,FATAL_B,HIGHWAY,INJURIES,dt
ACCID,,,,,,,,,,,,,,
12918,12,Middlesex,1218,Plainsboro Twsp,NaN,State Highway 1,1.0,0.0,0.0,1.0,0.0,1,NaN,2024-01-01 01:56:00-05:00
12923,11,Mercer,1111,Trenton,NaN,State Highway 129,1.0,1.0,0.0,0.0,0.0,129,NaN,2024-01-03 12:14:00-05:00
12932,01,Atlantic,0112,Hamilton Twsp,NaN,Atlantic City Expressway MP 20.5,1.0,1.0,0.0,0.0,0.0,446,0.0,2024-01-05 09:14:00-05:00
12933,02,Bergen,0248,Ramsey Boro,East Oak St,East Oak St,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-01-05 14:12:00-05:00
13362,07,Essex,0701,Belleville Town,NaN,County 647 MP .76,1.0,0.0,0.0,1.0,0.0,647,NaN,2024-01-05 18:08:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13439,12,Middlesex,1225,Woodbridge Twsp,NaN,Garden State Parkway MP 131.1,3.0,1.0,2.0,0.0,0.0,444,NaN,2024-09-14 23:55:00-04:00
13440,03,Burlington,0334,Southampton Twsp,NaN,State Highway 206 MP 23.3,1.0,1.0,0.0,0.0,0.0,206,NaN,2024-09-15 12:32:00-04:00
13441,18,Somerset,1811,Manville Boro,NaN,County 533,1.0,1.0,0.0,0.0,0.0,533,NaN,2024-09-15 17:33:00-04:00


Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

,driver,passenger,pedestrian,cyclist,crashes
county,,,,,
Atlantic,16,3,6,0,24
Bergen,15,1,4,0,20
Burlington,17,5,5,0,25
Camden,15,2,9,1,26
Cape May,7,3,0,0,9
Cumberland,8,2,3,0,12
Essex,10,3,19,1,33
Gloucester,15,4,3,0,19
Hudson,5,1,4,1,10


Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

71.6% through the year, 28.4% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

,,prv_ytd,prv_end,cur_ytd
county,type,,,
Atlantic,driver,20,31,16
Bergen,driver,13,14,15
Burlington,driver,24,31,17
Camden,driver,13,19,15
Cape May,driver,4,8,7
...,...,...,...,...
Salem,crashes,11,14,12
Somerset,crashes,12,18,8
Sussex,crashes,10,12,8


In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type,crashes,cyclist,driver,passenger,pedestrian
county,,,,,
Atlantic,38,0,25,5,10
Bergen,31,0,16,1,11
Burlington,35,0,23,5,11
Camden,36,1,22,3,12
Cape May,18,0,13,4,0
Cumberland,17,0,10,3,6
Essex,48,1,15,10,27
Gloucester,34,0,22,10,8
Hudson,18,1,8,1,10


In [9]:
projected.to_csv(PROJECTED_CSV)